In [1]:
import numpy as np
import tensorflow as tf

In [2]:
class LongShortTermMemoryModel:
    def __init__(self, encoding_size):
        # Model constants
        cell_state_size = 128

        # Cells
        cell = tf.contrib.rnn.BasicLSTMCell(cell_state_size)

        # Model input
        self.batch_size = tf.placeholder(tf.int32, [])  # Needed by cell.zero_state call, and can be dependent on usage (training or generation)
        self.x = tf.placeholder(tf.float32, [None, None, encoding_size])  # Shape: [batch_size, max_time, encoding_size]
        self.y = tf.placeholder(tf.float32, [None, None, encoding_size])  # Shape: [batch_size, max_time, encoding_size]
        self.in_state = cell.zero_state(self.batch_size, tf.float32)  # Can be used as either an input or a way to get the zero state

        # Model variables
        W = tf.Variable(tf.random_normal([cell_state_size, encoding_size]))
        b = tf.Variable(tf.random_normal([encoding_size]))

        # Model operations
        lstm, self.out_state = tf.nn.dynamic_rnn(cell, self.x, initial_state=self.in_state)  # lstm has shape: [batch_size, max_time, cell_state_size]

        # Logits, where tf.einsum multiplies a batch of txs matrices (lstm) with W
        logits = tf.nn.bias_add(tf.einsum('bts,se->bte', lstm, W), b)  # b: batch, t: time, s: state, e: encoding

        # Predictor
        self.f = tf.nn.softmax(logits)

        # Cross Entropy loss
        self.loss = tf.losses.softmax_cross_entropy(self.y, logits)

In [3]:
char_encodings = [
    [1, 0, 0, 0, 0, 0, 0, 0],  # ' '
    [0, 1, 0, 0, 0, 0, 0, 0],  # 'h'
    [0, 0, 1, 0, 0, 0, 0, 0],  # 'e'
    [0, 0, 0, 1, 0, 0, 0, 0],  # 'l'
    [0, 0, 0, 0, 1, 0, 0, 0],  # 'o'
    [0, 0, 0, 0, 0, 1, 0, 0],  # 'w'
    [0, 0, 0, 0, 0, 0, 1, 0],  # 'r'
    [0, 0, 0, 0, 0, 0, 0, 1],  # 'd'
]
encoding_size = np.shape(char_encodings)[1]

index_to_char = [' ', 'h', 'e', 'l', 'o', 'w', 'r', 'd']

x_train = [char_encodings[0], char_encodings[1], char_encodings[2], char_encodings[3], char_encodings[3], char_encodings[4], char_encodings[0], char_encodings[5], char_encodings[4], char_encodings[6], char_encodings[3], char_encodings[7]]  # ' hello'
y_train = [char_encodings[1], char_encodings[2], char_encodings[3], char_encodings[3], char_encodings[4], char_encodings[0], char_encodings[5], char_encodings[4], char_encodings[6], char_encodings[3], char_encodings[7], char_encodings[0]]  # 'hello '

In [4]:
model = LongShortTermMemoryModel(encoding_size)

# Training: adjust the model so that its loss is minimized
minimize_operation = tf.train.RMSPropOptimizer(0.05).minimize(model.loss)

# Create session object for running TensorFlow operations
session = tf.Session()

# Initialize tf.Variable objects
session.run(tf.global_variables_initializer())

# Initialize model.in_state
zero_state = session.run(model.in_state, {model.batch_size: 1})

W0917 13:06:29.448534 139808889096000 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0917 13:06:29.449350 139808889096000 deprecation.py:323] From <ipython-input-2-b5a3ce92a02a>:7: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0917 13:06:29.478246 139808889096000 deprecation.py:323] From <ipython-input-2-b5a3ce92a02a>:20: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please

In [5]:
for epoch in range(500):
    session.run(minimize_operation, {model.batch_size: 1, model.x: [x_train], model.y: [y_train], model.in_state: zero_state})

    if epoch % 10 == 9:
        print("epoch", epoch)
        print("loss", session.run(model.loss, {model.batch_size: 1, model.x: [x_train], model.y: [y_train], model.in_state: zero_state}))

        # Generate characters from the initial characters ' h'
        state = session.run(model.in_state, {model.batch_size: 1})
        text = ' h'
        y, state = session.run([model.f, model.out_state], {model.batch_size: 1, model.x: [[char_encodings[0]]], model.in_state: state})  # ' '
        y, state = session.run([model.f, model.out_state], {model.batch_size: 1, model.x: [[char_encodings[1]]], model.in_state: state})  # 'h'
        text += index_to_char[y.argmax()]
        for c in range(50):
            y, state = session.run([model.f, model.out_state], {model.batch_size: 1, model.x: [[char_encodings[y.argmax()]]], model.in_state: state})
            text += index_to_char[y[0].argmax()]
        print(text)

epoch 9
loss 1.5750618
 hllo lo   l   wl   l    l     l     l     l      l  
epoch 19
loss 1.1620361
 hello wo r wd   wd  wd   wd  wd  wd   wd  wd  wd  wd
epoch 29
loss 0.97323537
 helo wor d  helo  wor d  helo  wor d  helo  wor d  h
epoch 39
loss 0.56165713
 helo world d wrd d held word d held word d held word
epoch 49
loss 0.17030548
 hello world hello world hello world hello world hell
epoch 59
loss 0.06155604
 hello world  hello world  hello world  hello world  
epoch 69
loss 0.028967798
 hello world  hello world  hello world  hello world  
epoch 79
loss 0.014044245
 hello world  hello world  hello world  hello world  
epoch 89
loss 0.0069261305
 hello world  hello world  hello world  hello world  
epoch 99
loss 0.0034620382
 hello world  hello world  hello world  hello world  
epoch 109
loss 0.0017523557
 hello world  hello world  hello world  hello world  
epoch 119
loss 0.0008979179
 hello world  hello world  hello world  hello world  
epoch 129
loss 0.00046551204
 hello world  

In [7]:
session.close()